# Features 
| Column Name       | Description                                                                 | Type      | Mathematical Basis / Formula |
|--------------------|------------------------------------------------------------------------------|------------|--------------------------------|
| **Subject**        | Participant identifier.                                                     | int        | — |
| **SedationLabel**  | Sedation level label: Baseline, Mild, Moderate, Recovery.                   | string     | — |
| **Band**           | EEG frequency band (Delta, Theta, Alpha, Beta, Gamma).                      | string     | Defined by frequency bounds $(f_{min}, f_{max})$ |
| **mean_degree**    | Average weighted node degree — mean sum of connection weights per node.     | float      | $k_i = \sum_j w_{ij}$, $\langle k \rangle = \frac{1}{N} \sum_i k_i$ |
| **clustering**     | Weighted clustering coefficient — measures local interconnectedness.         | float      | $C = \frac{1}{N} \sum_i \frac{(W^{1/3})_{ij}(W^{1/3})_{jk}(W^{1/3})_{ki}}{k_i(k_i - 1)}$ |
| **path_length**    | Characteristic path length — mean shortest path between all node pairs.     | float      | $L = \frac{1}{N(N-1)} \sum_{i \ne j} d_{ij}$, $d_{ij} = 1 / w_{ij}$ |
| **small_worldness**| Small-worldness coefficient — ratio of normalized clustering to path length. | float      | $\sigma = \frac{C / C_{rand}}{L / L_{rand}}$ |
| **Propofol_ugL**   | Propofol plasma concentration at recording time (µg/L).                     | float      | — |
| **RT_ms**          | Mean reaction time (ms) in behavioral task.                                 | float      | — |
| **Correct**        | Number of correct responses (max = 40).                                     | int        | — |
| **Responsiveness**      | Behavioral responsiveness classification: 1 = responsive, 0 = drowsy/unresponsive. Based on overlap of binomial confidence intervals between baseline and moderate sedation hit-rate distributions.         | int        | $R = 1 \text{ if } CI_{\text{mod}} \cap CI_{\text{base}} \neq \emptyset,\ 0 \text{ otherwise}$ |

In [1]:
# --- Project path setup ---
import sys
from pathlib import Path

ROOT = Path(__file__).resolve().parents[1] if "__file__" in globals() else Path.cwd().parent
sys.path.append(str(ROOT / "src"))

In [3]:
import numpy as np
import pandas as pd
import mne
from pathlib import Path
from transformation import compute_band_ratios
from normalization import normalize_within_subject
from connectivity import compute_dwpli, compute_wpli_per_epoch
from graph_metrics import compute_graph_metrics, compute_graph_metrics_epochs
import os
from tqdm import tqdm

FREQ_BANDS = {
    "delta": (1, 4),
    "theta": (4, 8),
    "alpha": (8, 13),
    "beta": (13, 30),
    "gamma": (30, 45),
}

MANIFEST_PATH = Path("../data/data_derivatives/manifests/manifest.csv")
manifest = pd.read_csv(MANIFEST_PATH)

# --- Resolve paths relative to the project root ---
# (Assumes notebook is inside /notebooks)
PROJECT_ROOT = Path.cwd().parent
manifest["SetPath"] = manifest["SetPath"].apply(lambda p: (PROJECT_ROOT / p).resolve())

# --- Subset for testing ---
subset_manifest = manifest[manifest["Subject"].isin([1, 2])]


In [ ]:
# Version for epochs averaged
records = []

sedation_map = {
    "Baseline": 1,
    "Mild": 2,
    "Moderate": 3,
    "Recovery": 4
}

for idx in range(len(manifest)):
    row = manifest.iloc[idx]
    set_path = Path(row["SetPath"])
    
    if not set_path.exists():
        print(f"File not found: {set_path}")
        continue

    # Load EEG epochs
    try:
        epochs = mne.io.read_epochs_eeglab(set_path, verbose="error")
    except Exception as e:
        print(f"Could not load {set_path.name}: {e}")
        continue

    # Loop over frequency bands
    for band, (fmin, fmax) in FREQ_BANDS.items():
        try:
            con_matrix = compute_dwpli(epochs, fmin, fmax)

            if np.isnan(con_matrix).all() or con_matrix.shape[0] == 0:
                raise ValueError(f"Empty or NaN matrix in {band}")

            metrics = compute_graph_metrics(con_matrix, n_rand=10)

            record = {
                "Subject": row["Subject"],
                "SedationLabel": row["SedationLabel"],
                "SedationLevel": sedation_map.get(row["SedationLabel"], None),
                "Band": band,
                **metrics,
                "Propofol_ugL": row["Propofol_ugL"],
                "RT_ms": row["RT_ms"],
                "Correct": row["Correct"],
            }
            records.append(record)

        except Exception as e:
            print(f"Error in {row['BaseName']} ({band}): {type(e).__name__}: {e}")
            continue

df_metrics = pd.DataFrame(records)
df_metrics.to_csv("../data/data_derivatives/features.csv", index=False)

In [ ]:
# --- Version for per-epoch metrics (not averaged) ---
records = []

sedation_map = {
    "Baseline": 1,
    "Mild": 2,
    "Moderate": 3,
}

# Directory to cache wPLI matrices
save_dir = Path("../data/data_derivatives/connectivity_matrices")
save_dir.mkdir(parents=True, exist_ok=True)

# --- Outer loop: subjects ---
for idx in tqdm(range(len(manifest)), desc="Subjects", position=0):
    row = manifest.iloc[idx]
    set_path = Path(row["SetPath"])
    label = row["SedationLabel"]

    if label == "Recovery":
        continue

    if not set_path.exists():
        print(f"File not found: {set_path}")
        continue

    # --- Load EEG epochs ---
    try:
        epochs = mne.io.read_epochs_eeglab(set_path, verbose="error")
    except Exception as e:
        print(f"Could not load {set_path.name}: {e}")
        continue

    # --- Inner loop: frequency bands ---
    for band, (fmin, fmax) in tqdm(FREQ_BANDS.items(),
                                   desc=f"Freq bands (Subject {row['Subject']})",
                                   leave=False,
                                   position=1):
        try:
            # Cached file path for this subject × band
            save_path = save_dir / f"{row['Subject']}_{label}_{band}_wpli.npy"


            # Load or compute dwPLI matrices
            if save_path.exists():
                con_matrices = np.load(save_path, allow_pickle=True)
            else:
                con_matrices = compute_wpli_per_epoch(epochs, fmin, fmax)
                np.save(save_path, con_matrices)

            # Loop over epochs
            for ep_idx, con_matrix in enumerate(con_matrices):
                if np.isnan(con_matrix).all() or con_matrix.shape[0] == 0:
                    raise ValueError(f"Empty or NaN matrix in {band}, epoch {ep_idx}")

                metrics = compute_graph_metrics_epochs(con_matrix, n_rand=10)

                record = {
                    "Subject": row["Subject"],
                    "Epoch": ep_idx,
                    "SedationLabel": label,
                    "SedationLevel": sedation_map.get(label, None),
                    "Band": band,
                    **metrics,
                    "Propofol_ugL": row["Propofol_ugL"],
                    "RT_ms": row["RT_ms"],
                    "Correct": row["Correct"],
                }
                records.append(record)

        except Exception as e:
            print(f"Error in {row['BaseName']} ({band}): {type(e).__name__}: {e}")
            continue

# --- Combine and save ---
df_metrics = pd.DataFrame(records)
df_metrics.to_csv("../data/data_derivatives/features_per_epoch.csv", index=False)

In [ ]:
# Transformation
df_metrics = pd.read_csv("../data/data_derivatives/features_per_epoch.csv")

df_ratios = compute_band_ratios(
    df_metrics,
    output_path="../data/data_derivatives/features_band_ratios.csv"
)

In [ ]:
# Merge on subject + sedation level
df_combined = pd.merge(
    df_metrics,
    df_ratios,
    on=["Subject", "SedationLevel"],
    how="left"
)

# Save for reproducibility
df_combined.to_csv("../data/data_derivatives/features_ratio_metrics.csv", index=False)

df_combined.head()


In [ ]:

df_delta = normalize_within_subject(df_combined)
df_delta.to_csv("../data/data_derivatives/features_within_subject.csv", index=False)